In [1]:
# %load some_orm.py
import json

from typing import List, Dict, Any, Iterator
# from pprint import pprint as pp


def select(*field_names: str) -> callable:
    """
    Create a function for selecting the fields of data in 'query'.

    :param field_names: The list of fields for selection
    :return: The function for selecting the fields
    """
    def inner(data: List[Dict[str, Any]]) -> Iterator:
        return map(
            lambda row: {key: row[key] for key in row if key in field_names},
            data)

    return inner


def field_filter(field_name: str, *values: str) -> callable:
    """
    Create a function for filtering the data in 'query'.

    :param field_name: The name of field for filtering
    :param values: The list of values for this field
    :return: The function
    """
    def inner(data: List[Dict[str, Any]]) -> Iterator:
        return filter(lambda row: row[field_name] in values, data)

    return inner


def query(filepath: str,
          selection: callable,
          *filters: callable
          ) -> List[Dict[str, Any]]:
    """
    Select fields in each row of the 'data', leave only the the fields
    which return 'selection' and filtered them via calls of 'filters'.

    :param filepath: The path to the json-file
    :param selection: The function, which select the fields of interest
    :param filters: The functions for filtering the fields
    :return: The list of filtered records from 'data', including only fields of interest
    """

    with open(filepath, 'r') as f:
        data = json.load(f)

    for function in [selection, *filters]:
        data = function(data)

    return list(data)


In [ ]:
# %load test_some_orm_with_fixtures.py
import json
import os
import pytest

from some_orm import query, select, field_filter


@pytest.fixture(scope='module')
def prepare_data():
    friends = [
        {'name': 'Сэм', 'gender': 'Мужской', 'sport': 'Баскетбол', 'age': 28},
        {'name': 'Эмили', 'gender': 'Женский', 'sport': 'Волейбол', 'age': 22},
        {'name': 'Роберт', 'gender': 'Мужской', 'sport': 'Баскетбол', 'age': 25},
        {'name': 'Амалия', 'gender': 'Женский', 'sport': 'Теннис', 'age': 31},
        {'name': 'Роберт', 'gender': 'Мужской', 'sport': 'Хоккей', 'age': 20},
        {'name': 'Алекс', 'gender': 'Мужской', 'sport': 'Волейбол', 'age': 35},
        {'name': 'Крис', 'gender': 'Мужской', 'sport': 'Теннис', 'age': 27}
    ]
    with open('data.json', 'w') as f:
        json.dump(friends, f)

    yield 'data.json'

    os.remove('data.json')


def test_some_orm_positive(prepare_data):
    actual = query(
        prepare_data,
        select('name', 'gender', 'sport'),
        field_filter('sport', 'Баскетбол', 'Волейбол'),
        field_filter('gender', 'Мужской'),
    )
    expected = [
        {'gender': 'Мужской', 'name': 'Сэм', 'sport': 'Баскетбол'},
        {'gender': 'Мужской', 'name': 'Роберт', 'sport': 'Баскетбол'},
        {'gender': 'Мужской', 'name': 'Алекс', 'sport': 'Волейбол'}
    ]
    assert actual == expected


def test_nothing_found_positive(prepare_data):
    actual = query(
        prepare_data,
        select('name', 'gender', 'sport'),
        field_filter('sport', 'Хоккей'),
        field_filter('gender', 'Женский'),
    )
    assert actual == []


def test_unknown_field(prepare_data):
    with pytest.raises(Exception) as e_info:
        query(
            prepare_data,
            select('name', 'surname', 'sport'),
            field_filter('sport', 'Хоккей'),
            field_filter('gender', 'Женский'),
        )
    assert 'KeyError' in str(e_info)


In [8]:
!pytest test_some_orm_with_fixtures.py

============================= test session starts =============================
platform win32 -- Python 3.7.4, pytest-5.2.1, py-1.8.0, pluggy-0.13.0
rootdir: F:\!EPAM\HW_Lesson11
plugins: arraydiff-0.3, cov-2.10.0, doctestplus-0.4.0, openfiles-0.4.0, remotedata-0.3.2
collected 3 items

test_some_orm_with_fixtures.py ...                                       [100%]

============================== 3 passed in 0.08s ==============================


In [9]:
!coverage run -m pytest test_some_orm_with_fixtures.py

============================= test session starts =============================
platform win32 -- Python 3.7.4, pytest-5.2.1, py-1.8.0, pluggy-0.13.0
rootdir: F:\!EPAM\HW_Lesson11
plugins: arraydiff-0.3, cov-2.10.0, doctestplus-0.4.0, openfiles-0.4.0, remotedata-0.3.2
collected 3 items

test_some_orm_with_fixtures.py ...                                       [100%]

============================== 3 passed in 0.09s ==============================


In [10]:
!coverage report

Name                             Stmts   Miss  Cover
----------------------------------------------------
some_orm.py                         16      0   100%
test_some_orm_with_fixtures.py      21      0   100%
----------------------------------------------------
TOTAL                               37      0   100%
